In [ ]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.sklearn

from joblib import load
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from mlflow.models.signature import infer_signature

In [ ]:
# loading data

train = pd.read_csv("../data/processed/train.csv")
train.head(3)

X_train = train.drop(['Unnamed: 0', 'PATHOLOGY'], axis=1)
y_train = train['PATHOLOGY']

test = pd.read_csv("../data/processed/test.csv")
test.head(3)

X_test = test.drop(['Unnamed: 0', 'PATHOLOGY'], axis=1)
y_test = test['PATHOLOGY']
labels=y_test.unique()

In [5]:
# dummy classifier as a baseline

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
DummyClassifier(strategy='most_frequent')
dummy_clf.predict(X_train)
dummy_clf.score(X_test, y_test)

0.06498970482200864

In [6]:
# log reg model

clf = load("../models/baseline.joblib") 
y_pred = clf.predict(X_test)
y_probas = clf.predict_proba(X_test)
clf.score(X_test, y_test)

0.9930646923711616

In [7]:
score = clf.score(X_test, y_test)
print("Score: %s" % score)
mlflow.log_metric("score", score)
predictions = clf.predict(X_test)
signature = infer_signature(X_test, predictions)
mlflow.sklearn.log_model(clf, "model", signature=signature)
print("Model saved in run %s" % mlflow.active_run().info.run_uuid)

Score: 0.9930646923711616


/Users/fatima/miniforge3/envs/ml/lib/python3.10/site-packages/mlflow/models/signature.py:137: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input)


Model saved in run ddcc3f9863cc45d4a80fbcf722fc7bb0


/Users/fatima/miniforge3/envs/ml/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [9]:
# ! mlflow ui